In [2]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [3]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [5]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [6]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0529.csv	    dataframe_result_530_UK.csv
 a0530.csv	    dataframe_result_531_UK.csv
 a0531.csv	    dataframe_result_csv
 a0601.csv	    dataframe_result_csv_EU
 a0602.csv	    dataframe_result_csv_UK
 a0603.csv	    Github
 a0604.csv	    heatmap529_UK.ipynb
 a0605.csv	    heatmap530.ipynb
 a0606.csv	    Heat_Map_Online_language.ipynb
 a0607.csv	    __MACOSX
 a0608.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [7]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
ct=pd.read_csv('a0529-a0611_UK/a0603.csv',encoding='ISO-8859-1')

In [9]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0529.csv	    dataframe_result_530_UK.csv
 a0530.csv	    dataframe_result_531_UK.csv
 a0531.csv	    dataframe_result_csv
 a0601.csv	    dataframe_result_csv_EU
 a0602.csv	    dataframe_result_csv_UK
 a0603.csv	    Github
 a0604.csv	    heatmap529_UK.ipynb
 a0605.csv	    heatmap530.ipynb
 a0606.csv	    Heat_Map_Online_language.ipynb
 a0607.csv	    __MACOSX
 a0608.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [10]:
###
#ct=pd.read_csv('a0601.csv')

In [11]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,419544961466200_2985172334903437,Facebook,2020-06-03 23:59:54,2021-08-24 06:13:28,link,George Floyd Updates: 3 More Officers Charged;...,nytimes.com,As unrest over race and policing roils the nat...,Finally! What say you? https://www.nytimes.com...,2.0,https://www.nytimes.com/2020/06/03/us/live-geo...,https://www.facebook.com/419544961466200/posts...,183197,1.720588,1.0,78,13,5,16,5,0,0,0,0,2,28,5,14,6,2,5,3,5,0,2,403956,Victor Willis,officialvictorwillis,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Official Facebook Page of VICTOR WILLIS, lead ...",True,2013-02-17 05:01:34,en,0,403956|2985172334903437,{},{'Virginia': 1},{}
1,1,719648111502249_1941990192601362,Facebook,2020-06-03 23:59:53,2021-07-10 00:16:35,link,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,George Floyd died at the knee of a police offi...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/719648111502249/posts...,1729,-30.000000,1.0,0,0,0,0,0,0,0,0,0,0,5,2,4,2,1,0,1,0,0,1,4685612,Standard-Examiner Sports,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,4685612|1941990192601362,{},{'Minnesota': 1},{}
2,2,229748831408114_278454989870831,Facebook,2020-06-03 23:59:53,2021-04-03 03:49:13,status,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,It is so hard to figure out how to show up rig...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/groups/22974883140811...,4784,-1.341463,1.0,26,0,4,11,0,0,0,0,0,10,17,4,6,10,5,7,5,1,0,6,10735585,COVID-19 Acts of Grace and Generosity,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,10735585|278454989870831,{},{'Minnesota': 1},{}
3,3,1250734815048111_3089596414495266,Facebook,2020-06-03 23:59:50,2021-08-16 19:32:51,link,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",**This great man was a lifelong best friend of...,2.0,https://www.espn.com/college-football/story/_/...,https://www.facebook.com/groups/12507348150481...,28550,69.038462,1.0,62,673,133,3,11,0,672,241,0,7,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7838630|3089596414495266,{'Indianapolis': 1},{'Indianapolis': 1},{'Indianapolis': 1}
4,4,358599564868959_632768947452018,Facebook,2020-06-03 23:59:49,2021-07-04 09:11:04,photo,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",Handcuffed. Face Down. Knee on his neck. They ...,1.0,https://www.facebook.com/photo.php?fbid=102235...,https://www.facebook.com/groups/35859956486895...,4798,1.340659,1.0,91,96,22,2,1,0,96,58,0,2,136,35,22,37,4,19,6,14,0,4,7504729,Build Back Better Florida,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,faceb

In [12]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [13]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [14]:
###
ct_ner = ct

In [15]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'Virginia': 1},{}
1,{},{'Minnesota': 1},{}
2,{},{'Minnesota': 1},{}
3,{'Indianapolis': 1},{'Indianapolis': 1},{'Indianapolis': 1}
4,{'America': 1},{'Indianapolis': 1},{'Indianapolis': 1}


In [16]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [17]:
ct_ner['NER-msg']

0                                              []
1                                              []
2                                              []
3                                  [Indianapolis]
4                                       [America]
                           ...                   
50763    [Minneapolis, South Carolina, Minnesota]
50764                                          []
50765                                          []
50766                                          []
50767                                          []
Name: NER-msg, Length: 50768, dtype: object

In [18]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [19]:
ct_ner['NER']

0                                               [Virginia]
1                                              [Minnesota]
2                                              [Minnesota]
3               [Indianapolis, Indianapolis, Indianapolis]
4                    [America, Indianapolis, Indianapolis]
                               ...                        
50763    [Minneapolis, South Carolina, Minnesota, South...
50764                                     [South Carolina]
50765                                     [South Carolina]
50766                             [Minneapolis, Minnesota]
50767                                         [Morristown]
Name: NER, Length: 50768, dtype: object

In [20]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [21]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [22]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,419544961466200_2985172334903437,Facebook,2020-06-03 23:59:54,2021-08-24 06:13:28,link,George Floyd Updates: 3 More Officers Charged;...,nytimes.com,As unrest over race and policing roils the nat...,Finally! What say you? https://www.nytimes.com...,2.0,https://www.nytimes.com/2020/06/03/us/live-geo...,https://www.facebook.com/419544961466200/posts...,183197,1.720588,1.0,78,13,5,16,5,0,0,0,0,2,28,5,14,6,2,5,3,5,0,2,403956,Victor Willis,officialvictorwillis,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Official Facebook Page of VICTOR WILLIS, lead ...",True,2013-02-17 05:01:34,en,0,403956|2985172334903437,[],[Virginia],[],[virginia]
1,1,719648111502249_1941990192601362,Facebook,2020-06-03 23:59:53,2021-07-10 00:16:35,link,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,George Floyd died at the knee of a police offi...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/719648111502249/posts...,1729,-30.000000,1.0,0,0,0,0,0,0,0,0,0,0,5,2,4,2,1,0,1,0,0,1,4685612,Standard-Examiner Sports,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,4685612|1941990192601362,[],[Minnesota],[],[minnesota]
2,2,229748831408114_278454989870831,Facebook,2020-06-03 23:59:53,2021-04-03 03:49:13,status,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,It is so hard to figure out how to show up rig...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/groups/22974883140811...,4784,-1.341463,1.0,26,0,4,11,0,0,0,0,0,10,17,4,6,10,5,7,5,1,0,6,10735585,COVID-19 Acts of Grace and Generosity,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,10735585|278454989870831,[],[Minnesota],[],[minnesota]
3,3,1250734815048111_3089596414495266,Facebook,2020-06-03 23:59:50,2021-08-16 19:32:51,link,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",**This great man was a lifelong best friend of...,2.0,https://www.espn.com/college-football/story/_/...,https://www.facebook.com/groups/12507348150481...,28550,69.038462,1.0,62,673,133,3,11,0,672,241,0,7,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7838630|3089596414495266,[Indianapolis],[Indianapolis],[Indianapolis],[indianapolis]
4,4,358599564868959_632768947452018,Facebook,2020-06-03 23:59:49,2021-07-04 09:11:04,photo,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",Handcuffed. Face Down. Knee on his neck. They ...,1.0,https://www.facebook.com/photo.php?fbid=102235...,https://www.facebook.com/groups/35859956486895...,4798,1.340659,1.0,91,96,22,2,1,0,96,58,0,2,136,35,22,37,4,19,6,14,0,4,7504729,Build Back Better Florida,standardexsports,https://scontent-bos3-1.xx.fbcd

In [23]:
ct_ner['NER-1']=ct_ner['NER']

In [24]:
ct_ner['NER-1'][1]

['minnesota']

In [25]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 5.2 MB/s 


In [26]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [27]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,419544961466200_2985172334903437,Facebook,2020-06-03 23:59:54,2021-08-24 06:13:28,link,George Floyd Updates: 3 More Officers Charged;...,nytimes.com,As unrest over race and policing roils the nat...,Finally! What say you? https://www.nytimes.com...,2.0,https://www.nytimes.com/2020/06/03/us/live-geo...,https://www.facebook.com/419544961466200/posts...,183197,1.720588,1.0,78,13,5,16,5,0,0,0,0,2,28,5,14,6,2,5,3,5,0,2,403956,Victor Willis,officialvictorwillis,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Official Facebook Page of VICTOR WILLIS, lead ...",True,2013-02-17 05:01:34,en,0,403956|2985172334903437,[],[Virginia],[],[virginia],[virginia]
1,1,719648111502249_1941990192601362,Facebook,2020-06-03 23:59:53,2021-07-10 00:16:35,link,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,George Floyd died at the knee of a police offi...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/719648111502249/posts...,1729,-30.000000,1.0,0,0,0,0,0,0,0,0,0,0,5,2,4,2,1,0,1,0,0,1,4685612,Standard-Examiner Sports,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,4685612|1941990192601362,[],[Minnesota],[],[minnesota],[minnesota]
2,2,229748831408114_278454989870831,Facebook,2020-06-03 23:59:53,2021-04-03 03:49:13,status,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,It is so hard to figure out how to show up rig...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/groups/22974883140811...,4784,-1.341463,1.0,26,0,4,11,0,0,0,0,0,10,17,4,6,10,5,7,5,1,0,6,10735585,COVID-19 Acts of Grace and Generosity,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,10735585|278454989870831,[],[Minnesota],[],[minnesota],[minnesota]
3,3,1250734815048111_3089596414495266,Facebook,2020-06-03 23:59:50,2021-08-16 19:32:51,link,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",**This great man was a lifelong best friend of...,2.0,https://www.espn.com/college-football/story/_/...,https://www.facebook.com/groups/12507348150481...,28550,69.038462,1.0,62,673,133,3,11,0,672,241,0,7,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7838630|3089596414495266,[Indianapolis],[Indianapolis],[Indianapolis],[indianapolis],[indianapolis]
4,4,358599564868959_632768947452018,Facebook,2020-06-03 23:59:49,2021-07-04 09:11:04,photo,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",Handcuffed. Face Down. Knee on his neck. They ...,1.0,https://www.facebook.com/photo.php?fbid=102235...,https://www.facebook.com/groups/35859956486895...,4798,1.340659,1.0,91,96,22,2,1,0,96,58,0,2,136,35,22,37,4,19,6,14,0,4,7504729,Build Back Better 

In [28]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [29]:
!ls

Counties_UK1.txt	       dataframe_mapping_0605.csv
Counties_UK.txt		       dataframe_mapping_0606.csv
Country.txt		       dataframe_mapping_0607.csv
dataframe0529_post_remove.csv  dataframe_mapping_0608.csv
dataframe0530_post_remove.csv  dataframe_mapping_0609.csv
dataframe0531_post_remove.csv  dataframe_mapping_0610.csv
dataframe0607_post_remove.csv  dataframe_mapping_0611.csv
dataframe0608_post_remove.csv  dataframe_mapping_remove_0529.csv
dataframe0609_post_remove.csv  dataframe_mapping_remove_0530.csv
dataframe0610_post_remove.csv  dataframe_mapping_remove_0531.csv
dataframe0611_post_remove.csv  dataframe_mapping_remove_0606.csv
dataframe_mapping	       dataframe_mapping_remove_0607.csv
dataframe_mapping_0529.csv     dataframe_mapping_remove_0608.csv
dataframe_mapping_0530.csv     dataframe_mapping_remove_0609.csv
dataframe_mapping_0531.csv     dataframe_mapping_remove_0611.csv
dataframe_mapping_0601.csv     dataframe_post
dataframe_mapping_0602.csv     Post-specific_UK.zip
datafra

In [30]:
ct_mapping=pd.read_csv('dataframe_mapping_0603.csv',encoding='ISO-8859-1')

In [31]:
len(ct_mapping['City'])

5088

In [32]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [33]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [34]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [35]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [38]:
ct_mapping.to_csv('dataframe_mapping_remove_0603.csv')

In [39]:
len(ct_ner['NER-1'])

50768

In [40]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [41]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,419544961466200_2985172334903437,Facebook,2020-06-03 23:59:54,2021-08-24 06:13:28,link,George Floyd Updates: 3 More Officers Charged;...,nytimes.com,As unrest over race and policing roils the nat...,Finally! What say you? https://www.nytimes.com...,2.0,https://www.nytimes.com/2020/06/03/us/live-geo...,https://www.facebook.com/419544961466200/posts...,183197,1.720588,1.0,78,13,5,16,5,0,0,0,0,2,28,5,14,6,2,5,3,5,0,2,403956,Victor Willis,officialvictorwillis,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Official Facebook Page of VICTOR WILLIS, lead ...",True,2013-02-17 05:01:34,en,0,403956|2985172334903437,[],[Virginia],[],[virginia],[nan]
1,1,719648111502249_1941990192601362,Facebook,2020-06-03 23:59:53,2021-07-10 00:16:35,link,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,George Floyd died at the knee of a police offi...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/719648111502249/posts...,1729,-30.000000,1.0,0,0,0,0,0,0,0,0,0,0,5,2,4,2,1,0,1,0,0,1,4685612,Standard-Examiner Sports,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,4685612|1941990192601362,[],[Minnesota],[],[minnesota],[nan]
2,2,229748831408114_278454989870831,Facebook,2020-06-03 23:59:53,2021-04-03 03:49:13,status,Weber State Athletics issues statements denoun...,standard.net,OGDEN â As Americans grapple with the issues...,It is so hard to figure out how to show up rig...,1.0,https://www.standard.net/sports/weber-state/we...,https://www.facebook.com/groups/22974883140811...,4784,-1.341463,1.0,26,0,4,11,0,0,0,0,0,10,17,4,6,10,5,7,5,1,0,6,10735585,COVID-19 Acts of Grace and Generosity,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,10735585|278454989870831,[],[Minnesota],[],[minnesota],[nan]
3,3,1250734815048111_3089596414495266,Facebook,2020-06-03 23:59:50,2021-08-16 19:32:51,link,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",**This great man was a lifelong best friend of...,2.0,https://www.espn.com/college-football/story/_/...,https://www.facebook.com/groups/12507348150481...,28550,69.038462,1.0,62,673,133,3,11,0,672,241,0,7,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,standardexsports,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"Stories, games and features for Weber State at...",False,2015-09-30 16:34:38,en,0,7838630|3089596414495266,[Indianapolis],[Indianapolis],[Indianapolis],[indianapolis],"[Indianapolis, Marion County, Indiana, United ..."
4,4,358599564868959_632768947452018,Facebook,2020-06-03 23:59:49,2021-07-04 09:11:04,photo,"Ex-Hoosier Beaty shot, killed in Indianapolis",espn.com,"Former Hoosier Chris Beaty, who was 38, was sh...",Handcuffed. Face Down. Knee on his neck. They ...,1.0,https://www.facebook.com/photo.php?fbid=102235...,https://www.facebook.com/groups/35859956486895...,4798,1.340659,1.0,91,96,22,2,1,0,96,58,0,2,136,35,22,37,4,19,6,14,0,4,750472

In [42]:
ct_ner['NER-1'][12]

['El Paso, El Paso County, Texas, 79903, United States',
 nan,
 'Minneapolis, Hennepin County, Minnesota, 55415:55454, United States']

In [43]:
ct_ner.to_csv('dataframe0603_post_remove.csv')